# **Install**

In [ ]:
# !pip3 install ax-platform

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Check GPU
import tensorflow as tf
tf.test.gpu_device_name()

# Imports

In [ ]:
# General
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statistics
import pandas as pd
sns.set()

# keras imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, InputLayer, Conv1D, Flatten, MaxPool1D, BatchNormalization, MaxPooling1D
from keras.layers import PReLU 
from keras.initializers import glorot_uniform
from keras import optimizers

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MinMaxScaler,StandardScaler
from sklearn import metrics
from sklearn.model_selection import KFold

# Ax-platform imports
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting

plt.rcParams.update({
                     'axes.titlesize'  : 22,
                     'xtick.labelsize' : 12,
                     'ytick.labelsize' : 12,
                     'axes.labelsize'  : 16
                     })

plt.rcParams['figure.figsize'] = (10,6) 
plt.rcParams["font.family"] = "Serif"
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
# plt.rcParams.keys()

# Data and Scaling Data

In [ ]:
colab_dir = '/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/data_cleaned_means.csv'
df = pd.read_csv(colab_dir)

print(df['target'].describe())
plt.figure(figsize=(8, 4))
sns.distplot(df['target'], color='g', bins=100, hist_kws={'alpha': 0.4})

df.head(1)

In [ ]:
############################ Setting up Data and Target ###################################################################
X = df.drop(columns=['Unnamed: 0','ID','location','target','temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press','location']) # ,'temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press', ,'index'
y = df['target']
###########################################################################################################################
y = np.log(y)
y = np.array(y)
y = y.reshape(-1,1)
# # scalery = StandardScaler().fit(y)
# scalery = MinMaxScaler().fit(y)
# y = scalery.transform(y)
############################ SPLITTING DATA ######### #####################################################################
val_per = 0.2
test_per = 0.1

validation = X.shape[0]*val_per
testing = X.shape[0]*test_per
training = X.shape[0] - validation - testing

X_remain, X_test, y_remain, y_test = train_test_split(X, y, test_size=test_per, random_state=42)
per = 1-(training/X_remain.shape[0])
X_train,X_val,y_train,y_val =  train_test_split(X_remain, y_remain, test_size=per, random_state=42)


scalerX_remain = StandardScaler().fit(X_remain)
X_remain = scalerX_remain.transform(X_remain)

scalerX_train = StandardScaler().fit(X_train)
scalerX_val = StandardScaler().fit(X_val)
scalerX_test = StandardScaler().fit(X_test)
X_train = scalerX_train.transform(X_train)
X_val = scalerX_val.transform(X_val)
X_test = scalerX_test.transform(X_test)





X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_remain = X_remain.reshape(X_remain.shape[0], X_remain.shape[1], 1)


# # DOING THIS FOR ZINDI SUBMISSION # COMMENT THIS OUT and UNCOMMENT ABOVE TO RUN ASSIGNMENT INVESTIGATION
# X_train,X_val,y_train,y_val =  train_test_split(X, y, test_size=0.3, random_state=42)

# scalerX_train = StandardScaler().fit(X_train)
# scalerX_val = StandardScaler().fit(X_val)
# X_train = scalerX_train.transform(X_train)
# X_val = scalerX_val.transform(X_val)
# scalerX = StandardScaler().fit(X)
# X = scalerX.transform(X)


# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
# X = X.reshape(X.shape[0], X.shape[1], 1)

print('X_train shape : ',X_train.shape)
print('X_val shape : ',X_val.shape)
# print('X shape : ',X.shape)
print('X_remain shape : ',X_remain.shape)
print('X_test shape : ',X_test.shape)

# Model Buidling

In [ ]:
def cnn(num_filters,num_kernels,pools,neurons):
  model_m = Sequential()
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(Conv1D(filters = num_filters, kernel_size = num_kernels, activation = 'relu', input_shape = (X_train.shape[1],X_train.shape[2])))
  model_m.add(MaxPooling1D(pool_size = pools))
  model_m.add(Flatten())
  model_m.add(Dense(neurons, activation = 'relu'))
  model_m.add(Dense(1,activation = 'linear'))
  return model_m

def cnn_cv_score(parametrization, weight = None):
  model = cnn(parametrization.get('num_filters'),
              parametrization.get('num_kernels'),
              parametrization.get('pools'),
              parametrization.get('neurons'))
  
  learning_rate = parametrization.get('learning_rate')
  batch_size = parametrization.get('batch_size')

  model.compile(optimizer = optimizers.RMSprop(learning_rate = learning_rate), loss='mse', metrics = ['mse'])
  history = model.fit(X_train, y_train, epochs = NUM_EPOCHS,batch_size = batch_size, validation_data = (X_val,y_val))

  last10_scores = np.array(history.history['val_mse'][-10:])
  mean = last10_scores.mean()
  sem = last10_scores.std()

  if np.isnan(mean):
    return 9999.0, 0.0

  return mean, sem


NUM_EPOCHS = 15

In [ ]:
parameters = [
              {
                  "name" : "num_filters",
                  "type" : "choice",
                  "values" : [16,32,64],
              },
              {
                  "name" : "num_kernels",
                  "type" : "choice",
                  "values" : [2,3],
              },
              {
                  "name" : "pools",
                  "type" : "choice",
                  "values" : [2,3,4,5],
              },
              {
                  "name": "neurons",
                  "type": "range",
                  # "bounds": [20,100],
                  "bounds": [70,95],
                  "value_type": "int",
              },
              {
                  "name": "batch_size",
                  "type": "choice",
                  # "values": [32, 64, 128, 256]
                  "values": [32,64]
              },
              {
                  "name" : "learning_rate",
                  "type" : "range",
                  "bounds" : [0.0001, 0.5],
                  "log_scale": True,
              }
]

init_notebook_plotting()
ax_client = AxClient()

In [ ]:
number_of_experiments = 80

ax_client.create_experiment(
    name="keras_experiment",
    parameters=parameters,
    objective_name='keras_cv',
    minimize=True)

def evaluate(parameters):
    return {"keras_cv": cnn_cv_score(parameters)}

for i in range(number_of_experiments + 1):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))
ax_client.get_trials_data_frame().sort_values('trial_index')

In [ ]:
best_parameters, values = ax_client.get_best_parameters()
# the best set of parameters.
best = []
for k in best_parameters.items():
  best.append(k)
  print(k)

print()

# the best score achieved.
means, covariances = values
print(means)
print(covariances)

In [ ]:
def build_best_cnn_model(best_parameters):  
  model = cnn(best_parameters['num_filters'], 
              best_parameters['num_kernels'], 
              best_parameters['pools'],
              best_parameters['neurons'])
  model.compile(optimizer = optimizers.RMSprop(learning_rate = best_parameters['learning_rate']), loss = 'mse', metrics = ['mse'])
  return model

In [ ]:
batch_size = best_parameters['batch_size']
NUM_EPOCHS = 10

model = build_best_cnn_model(best_parameters)
history = model.fit(X_train, y_train, epochs = NUM_EPOCHS,batch_size = batch_size, validation_data = (X_val,y_val))

In [ ]:
# Lift Chart
def chart_regression(pred,y,sort=True):
  t = pd.DataFrame({'pred':pred.flatten(), 'y': y.flatten()})
  if sort :
    t.sort_values(by = ['y'],inplace=True)
    plt.title('Lift Chart (Test Set)',y = 1.05)
    plt.plot(t['pred'].tolist(),label='prediction',linewidth = 0.8)
    plt.plot(t['y'].tolist(),label='expected',linewidth = 3)
    plt.xlabel('Sorted Test Set (Ascending Order)')
    plt.ylabel('Air Quality')
    plt.legend(loc = 'best',fontsize = 16)
    plt.show()

data   = X_test
labels = y_test

y_pred_no_ensemble = model.predict(data)
y_new_inverse_no_ensemble = np.exp(y_pred_no_ensemble)
y_real_inverse = np.exp(labels)
plt.figure(1)
plt.title('Prediction vs Actual (Test Set)',y = 1.05)
plt.scatter(y_real_inverse,y_new_inverse_no_ensemble,alpha = 0.6)
plt.plot([0,375],[0,375],'k--',linewidth = 3)
plt.xlabel('Test Set')
plt.ylabel('Predictions')
plt.xlim([0,375])
plt.ylim([0,375])

score = metrics.mean_squared_error(y_new_inverse_no_ensemble,y_real_inverse)
score_mae = metrics.median_absolute_error(y_new_inverse_no_ensemble,y_real_inverse)
print('MAE : ',score_mae)
print('RMSE : ',np.sqrt(score))



plt.figure(2)
chart_regression(y_new_inverse_no_ensemble,y_real_inverse)

# Ensemble Modelling

In [ ]:
NUM_EPOCHS = 50
batch_size = best_parameters['batch_size']

n_splits = 15
kfolds = KFold(n_splits = n_splits ,shuffle = True,random_state = 42)
kfolds.get_n_splits(X_remain)
# kfolds.get_n_splits(X) 

15

In [ ]:
scores = []
models = []
current_fold = 1
for train,test in kfolds.split(X_remain):
# for train,test in kfolds.split(X):
  print('commencing fold {}:'.format(current_fold))
  print('  preparing data...')
  Xt = X_remain[train]
  Yt = y_remain[train]
  Xv = X_remain[test]
  Yv = y_remain[test] 

  # Xt = X[train]
  # Yt = y[train]
  # Xv = X[test]
  # Yv = y[test] 

  print(len(train))
  print(len(test))
  print()

  # create, fit and test model
  print('  building model...')
  classifier = build_best_cnn_model(best_parameters)
  print('  fitting model...')
  classifier.fit(Xt, Yt, epochs=NUM_EPOCHS, batch_size=batch_size)
  print('  evaluating model...')
  score = classifier.evaluate(Xv, Yv, batch_size=batch_size)
  
  scores.append(score[-1])
  models.append(classifier)

  print('  fold {} mse: {}'.format(current_fold, score[-1]))
  current_fold += 1

  print('ensemble mse: {} % (+/- {} %)'.format(np.mean(scores), np.std(scores)))

In [ ]:
def ensemble_predict(data,ensemble_models):
  y_preds = []
  for index, classifier in enumerate(ensemble_models):
    print('getting predictions from model {}...'.format(index+1))
    y_pred = classifier.predict(data, batch_size = batch_size)
    y_pred = y_pred.flatten()
    y_preds.append(y_pred)
  
  predictions = np.mean(y_preds,axis = 0)
  return predictions.reshape(-1,1)

In [ ]:
data   = X_test
labels = y_test

y_pred_ensemble = ensemble_predict(data,models)
y_new_inverse_ensemble = np.exp(y_pred_ensemble)
y_real_inverse = np.exp(labels)

plt.figure(1)
plt.title('Prediction vs Actual (Test Set)',y = 1.05)
plt.scatter(y_real_inverse,y_new_inverse_no_ensemble,color = 'r',alpha = 0.6,label = 'CNN')
plt.scatter(y_real_inverse,y_new_inverse_ensemble,color = 'b',alpha = 0.6,label = 'Ensemble CNN')
plt.plot([0,375],[0,375],'k--',linewidth = 3)
plt.xlabel('Test Set')
plt.ylabel('Predictions')
plt.xlim([0,375])
plt.ylim([0,375])
plt.legend(loc='best',fontsize = 16)

score = metrics.mean_squared_error(y_new_inverse_ensemble,y_real_inverse)
score_mae = metrics.median_absolute_error(y_new_inverse_ensemble,y_real_inverse)
print('MAE : ',score_mae)
print('RMSE : ',np.sqrt(score))

plt.figure(2)
chart_regression(y_new_inverse_ensemble,y_real_inverse)

# Model Summary

In [ ]:
best_cnn = build_best_cnn_model(best_parameters)
best_cnn.summary()
print()
best_parameters

# **Zindi Testing**

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from matplotlib import pyplot as plt
from sklearn import preprocessing

colab_dir = '/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/Test.csv'
df_test = pd.read_csv(colab_dir)
df_test_dummy = pd.read_csv(colab_dir)

Train = df_test
# Used for periodic data checks
dummy_Train = df_test_dummy

# Convert Strings to Numerical Values
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
    
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    Train[feature]=Train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    dummy_Train[feature]=dummy_Train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])

# Encoding sensor location (A,B,C,D,E) as Numeric Values (0,1,2,3,4,5)
le = preprocessing.LabelEncoder()
Train['location'] = le.fit_transform(Train['location'])

def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x      
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

for col_name in tqdm(features):
    Train[col_name]=Train[col_name].apply(remove_nan_values)
for col_name in tqdm(features):
    Train = aggregate_features(Train,col_name)

# Setting NaN's to the mean of the list
type_data = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
for k in range(len(type_data)):
    for i in range(len(Train[type_data[k]])):
      # Train.at[i,type_data[k]]= pd.Series(dummy_Train[type_data[k]][i]).fillna(np.nanmedian(dummy_Train[type_data[k]][i])).tolist()
      Train.at[i,type_data[k]]= pd.Series(dummy_Train[type_data[k]][i]).fillna(np.nanmean(dummy_Train[type_data[k]][i])).tolist()

for x in range(121):
    Train["newtemp"+ str(x)] = Train.temp.str[x]
    Train["newprecip"+ str(x)] = Train.precip.str[x]
    Train["newrel_humidity"+ str(x)] = Train.rel_humidity.str[x]
    Train["newwind_dir"+ str(x)] = Train.wind_dir.str[x]
    Train["windspeed"+ str(x)] = Train.wind_spd.str[x]
    Train["atmospherepressure"+ str(x)] = Train.atmos_press.str[x]


df_zindi = Train
X_zindi = df_zindi.drop(columns=['ID','location','temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press','location'])
scalerX_zindi = StandardScaler().fit(X_zindi)
X_zindi = scalerX_test.transform(X_zindi)
X_zindi = X_zindi.reshape(X_zindi.shape[0], X_zindi.shape[1], 1)

y_zindi = ensemble_predict(X_zindi,models)
y_zindi = np.exp(y_zindi)

columns_ID = df_zindi['ID']

df = pd.DataFrame(data = y_zindi.flatten() ,columns=['target'])
df = df.reset_index()
del df['index']

df1 = columns_ID
df1 = df1.reset_index()
del df1['index']

submission = pd.concat([df1,df],axis = 1)

In [ ]:
submission.to_csv('/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/submission.csv', index = False)